# Phase Estimation Algorithm

QPE can estimate the phases $\phi_i$ of a unitary with
$$U =\sum_i e^{2\pi i \phi_i} |\lambda_i\rangle\langle \lambda_i|$$

Here we demonstrate this with an implementation in `pennylane`.

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

## Create a random Unitary

In [ ]:
def create_random_vec(dim):
    """Create random unit complex vector (|psi>) of shape (dim, 1)."""
    vec = np.random.rand(dim) + 1j * np.random.rand(dim)
    return (vec / np.linalg.norm(vec))[:, None]

def proj(vec):
    """Given state |psi>, this returns |psi><psi|."""
    return vec @ np.conj(vec).T

def create_unitary(phases):
    """Create a 2x2 unitary with given phases. Phases should be a 2-component array."""
    eigvec_1 = create_random_vec(2)
    eigvec_2 = (np.eye(2) - proj(eigvec_1)) @ create_random_vec(2) 
    eigvec_2 /= np.linalg.norm(eigvec_2)
    unitary = np.exp(2*np.pi*1j*phases[0]) * proj(eigvec_1) + np.exp(2*np.pi*1j*phases[1]) * proj(eigvec_2)

    # Check it's unitary
    assert np.allclose(np.eye(2), np.conj(unitary.T) @ unitary)
    return unitary

## Create the circuit

In [ ]:
device = qml.device("default.qubit")

def create_random_state():
    qml.RY(2*np.pi*np.random.rand(), wires=0)
    qml.RZ(2*np.pi*np.random.rand(), wires=0)

def qpe_circuit(unitary, estimation_wires):
    for wire in estimation_wires:
        qml.H(wires=wire)

    qml.ControlledSequence(qml.QubitUnitary(unitary, 0), control=estimation_wires)
    qml.adjoint(qml.QFT)(wires=estimation_wires)

@qml.qnode(device)
def circuit(unitary, estimation_wires):
    create_random_state()
    qpe_circuit(unitary, estimation_wires)
    return qml.probs(wires=estimation_wires)

In [ ]:
print(qml.draw(circuit)(np.eye(2), [1, 2, 3]))

## Run

In [ ]:
def view_results(results, phases, show_every=1):
    _, ax = plt.subplots()
    xs = np.arange(len(results))
    ax.bar(xs, results)
    
    qbs = int(np.log2(len(results)))
    _xticks = [f"{i/len(xs)}\n({np.binary_repr(i, qbs)})" for i in xs]
    plt.xticks(xs[::show_every], _xticks[::show_every], rotation=15)

    for true_phase in phases:
        plt.axvline(x=true_phase * len(results), c='k', ls='dashed')

    ax.set_xlabel("phase")
    ax.set_ylabel("probability")

    plt.show()

### Case 1: Phases can be represented exactly

In [ ]:
estimation_wires = range(1, 4)

phases = np.array([0.25, 0.875])
unitary = create_unitary(phases)
results = circuit(unitary, estimation_wires)

view_results(results, phases)

### Case 2: Phases can't be represented exactly

In [ ]:
estimation_wires = range(1, 6)

phases = np.array([0.24108735, 0.81843316])
print("Phases:", phases)

unitary = create_unitary(phases)
results = circuit(unitary, estimation_wires)

view_results(results, phases, 4)

## Phase estimation of Hamiltonian
$$H =\sum_i  \lambda_i |\lambda_i\rangle\langle \lambda_i|$$
$$U_T = e^{iHT} =\sum_i  e^{i\lambda_i T} |\lambda_i\rangle\langle \lambda_i|$$

Thus
$$ \lambda_i = \frac{2\pi \phi_i}{T} $$

In [ ]:
hamiltonian = np.random.uniform(-1, 1, (2, 2)) + 1j * np.random.uniform(-1, 1, (2, 2))
hamiltonian = 0.5 * (hamiltonian + np.conj(hamiltonian.T))
eigs = np.real(np.linalg.eigvals(hamiltonian))
eigs

In [ ]:
duration = 1.
unitary = expm(hamiltonian * 1j * duration)

In [ ]:
phases = eigs * duration / (2 * np.pi)
for i in range(2):
    if phases[i] < 0.:
        phases[i] = 1 + phases[i]
phases

In [ ]:
estimation_wires = range(1, 6)
results = circuit(unitary, estimation_wires)

view_results(results, phases, 4)